In [1]:
!pip install openai langchain langchain-openai langchain-community -q -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 945.2/945.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [2]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

KeyboardInterrupt: 

## Если используете ключ из курса, запустите эти ячейки 👇

In [3]:
from langchain_openai import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите API ключ··········


# Задание 2.2.7: ⚔️ Spam 🆚 Crypto

In [4]:
import pandas as pd
from langchain import PromptTemplate
from tqdm import tqdm

In [5]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/100_crypto_messages.csv')

In [6]:
# Напишите промпт, указав инструкцию, запрос и выходной индикатор
template = """...

Context: {text_input}

Question: ...

Answer: ...
"""

prompt_template = PromptTemplate(
    input_variables=["text_input"],
    template=template
)

In [7]:
spam_labels = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    spam_label = llm.invoke(prompt).content # Сохраняем ответ модели
    spam_labels.append(spam_label) # Добавляем ответ в список
    break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

  0%|          | 0/100 [00:02<?, ?it/s]


In [8]:
spam_labels

['Пожалуйста, уточните, что именно вас интересует. Вы хотите узнать больше о вакансии, условиях удаленной работы или о чем-то другом?']

In [ ]:
df['is_spam'] = spam_labels # Создаём новый столбец из ответов модели

In [ ]:
# Полученный файл можно найти, нажав в левом меню значек с папкой 📁
df.to_csv('2_2_7_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)

# Задание 2.2.8: 💼 ChatGPT - ваш ручной карьерный консультант

В данной задаче очень удобно использовать `OutputParser` (подробнее о нем в [ноутбуке](https://github.com/a-milenkin/LLM_practical_course/blob/main/notebooks/M2.2_LangChain_Prompting.ipynb)), но не обязательно!

In [9]:
import pandas as pd
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [10]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')

In [11]:
template = """Из следующего текста извлеки информацию:

job_title: ...

company: ...

salary: ...

tg: ...

grade: ...

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [12]:
job_title_schema = ResponseSchema(
    name="job_title",
    description="..."
)


company_schema = ResponseSchema(
    name="company",
    description="..."
)


salary_schema = ResponseSchema(
    name="salary",
    description="..."
)


tg_schema = ResponseSchema(
    name="tg",
    description="..."
)


grade_schema = ResponseSchema(
    name="grade",
    description="..."
)

response_schemas = [
    job_title_schema,
    company_schema,
    salary_schema,
    tg_schema,
    grade_schema
]

In [13]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

In [14]:
dict_list = [] # Список из словарей, где будем хранить полученную информацию

for i, text_input in enumerate(tqdm(df['text'])):
    try:
        messages = prompt.format_messages(
            text=text_input,
            format_instructions=format_instructions
        )
        response = llm.invoke(messages) # Ответ модели

        # Пытаемся распарсить ответ
        try:
            parsed_response = output_parser.parse(response.content)
            dict_list.append(parsed_response)
        except OutputParserException:
            # Если автоматический парсер не работает, пытаемся извлечь JSON вручную
            content = response.content.strip()

            # Удаляем возможные markdown обрамления
            if '```json' in content:
                content = content.split('```json')[1].split('```')[0].strip()
            elif content.startswith('```'):
                content = content.replace('```', '').strip()

            # Парсим JSON
            json_obj = json.loads(content)
            dict_list.append(json_obj)

    except Exception as e:
        print(f"Ошибка при обработке текста {i}: {e}")
        # Добавляем пустой словарь в случае ошибки
        dict_list.append({
            "job_title": None,
            "company": None,
            "salary": None,
            "tg": None,
            "grade": None
        })

 10%|█         | 5/50 [00:13<01:59,  2.65s/it]

Ошибка при обработке текста 4: name 'OutputParserException' is not defined


100%|██████████| 50/50 [01:45<00:00,  2.11s/it]


In [15]:
dict_list

[{'job_title': 'Senior Python developer',
  'company': 'Collectly',
  'salary': '6000-9000$',
  'tg': '@ann_gfio',
  'grade': 'senior'},
 {'job_title': 'Системный аналитик',
  'company': 'АКЦИОНЕРНОЕ ОБЩЕСТВО «ГЛАВНЫЙ НАУЧНЫЙ ИННОВАЦИОННЫЙ ВНЕДРЕНЧЕСКИЙ ЦЕНТР» (АО «ГНИВЦ»)',
  'salary': '320 000 - 370 000 рублей',
  'tg': '@NatalyaMaki',
  'grade': ''},
 {'job_title': 'Senior Database Administrator',
  'company': 'Match Systems',
  'salary': 'от 3000$ (определяется по результатам собеседования)',
  'tg': '@lex_kertis',
  'grade': 'Senior'},
 {'job_title': 'Руководитель отдела системного администрирования',
  'company': 'финтех компания',
  'salary': 'от 3000$',
  'tg': '@ResearcherRIT',
  'grade': 'регулярный пересмотр грейда и ЗП'},
 {'job_title': None,
  'company': None,
  'salary': None,
  'tg': None,
  'grade': None},
 {'job_title': 'Senior DevOps Engineer',
  'company': 'Mad Devs',
  'salary': 'до 5000$',
  'tg': '@recruiter_maddevs',
  'grade': 'Senior'},
 {'job_title': 'Системны

In [ ]:
dict_list = [] # Список из словарей, где будем хранить полученную информацию
for text_input in tqdm(df['text']):
    messages = prompt.format_messages(
        text=text_input,
        format_instructions=format_instructions
    )
    response = llm.invoke(messages) # Ответ модели
    dict_list.append(output_parser.parse(response.content)) # Переводим содержимое ответа модели в словарь и добавляем в список
    break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

In [16]:
result_df = pd.DataFrame(dict_list) # Создаем датафрейм из полученной информации
ans_df = pd.concat([df, result_df], axis=1) # Объединяем всё в один датафрейм
ans_df.to_csv('2_2_8_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)